## Imports

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


import random

import mlflow
print(mlflow.__version__)

import os

spark.conf.set("spark.databricks.mlflow.trackMLlib.enabled", "true")

RANDOM_SEED = 0
# Define experiment name with proper Databricks path
EXPERIMENT_NAME = "/Shared/team_2_2/mlflow-baseline"
# Create the experiment if it doesn't exist
try:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"Created new experiment with ID: {experiment_id}")
    else:
        print(f"Using existing experiment: {experiment.name}")
    mlflow.set_experiment(EXPERIMENT_NAME)
except Exception as e:
    print(f"Error with experiment setup: {e}")
    # Fallback to default experiment in workspace
    mlflow.set_experiment(f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/default")



## Helper Functions


In [0]:
def checkpoint_dataset(dataset, file_path):
    # Create base folder
    section = "2"
    number = "2"
    base_folder = f"dbfs:/student-groups/Group_{section}_{number}"
    dbutils.fs.mkdirs(base_folder)
    # Create subfolders if file_path contains directories
    full_path = f"{base_folder}/{file_path}.parquet"
    subfolder = "/".join(full_path.split("/")[:-1])
    dbutils.fs.mkdirs(subfolder)
    # Save dataset as a parquet file
    dataset.write.mode("overwrite").parquet(full_path)
    print(f"Checkpointed {file_path}")

# Things to keep in mind
- Predict two hours before
- Remove all the delay columns
- Are we only predicting departure delays or arrival delays also? For example, the pilot misses the landing, and has to circle back for 20 minutes. Should we solve for that? I don't think we should.

## Datasets - custom join

## Model Iterations baseline

In [0]:
%fs ls dbfs:/student-groups/Group_2_2/1_year_custom_joined/feature_eng

In [0]:
checkpoint_path = f"dbfs:/student-groups/Group_2_2"
# dataset_path = f"{checkpoint_path}/1_year_custom_joined/raw_data/training_splits"
dataset_path = f"{checkpoint_path}/1_year_custom_joined/feature_eng/training_splits"

# Read datasets from checkpoint
train_df = spark.read.parquet(f"{dataset_path}/train.parquet")
validation_df = spark.read.parquet(f"{dataset_path}/validation.parquet")

In [0]:
train_df.columns == validation_df.columns


# Feature Engineering (start)

In [0]:
# Feature Engineering

## CRS_DEP_TIME is local time so we can use this feature 
## But in order to use it, we have to convert it to minutes since midnight
## Otherwise the timing will be off b/c it's not true UTC

# train_df = train_df.\
#         withColumn("CRS_DEP_MINUTES", (F.floor(F.col("CRS_DEP_TIME") / 100) * 60 + (F.col("CRS_DEP_TIME") % 100))).\
#         drop("CRS_DEP_TIME").\
#         drop("CRS_ARR_TIME")

# validation_df = validation_df.\
#         withColumn("CRS_DEP_MINUTES", (F.floor(F.col("CRS_DEP_TIME") / 100) * 60 + (F.col("CRS_DEP_TIME") % 100))).\
#         drop("CRS_DEP_TIME").\
#         drop("CRS_ARR_TIME")


train_df = train_df.withColumn(
    "CRS_DEP_MINUTES", 
    (F.floor(F.col("CRS_DEP_TIME") / 100) * 60 + (F.col("CRS_DEP_TIME") % 100))
)

validation_df = validation_df.withColumn(
    "CRS_DEP_MINUTES", 
    (F.floor(F.col("CRS_DEP_TIME") / 100) * 60 + (F.col("CRS_DEP_TIME") % 100))
)


In [0]:
train_df = train_df.cache()
validation_df = validation_df.cache()

### Weather data
- drop rows if null

In [0]:
weather_feat_list = [
 'HourlyDryBulbTemperature',
 'HourlyDewPointTemperature',
 'HourlyRelativeHumidity',
 'HourlyAltimeterSetting',
 'HourlyVisibility',
 'HourlyStationPressure',
 'HourlyWetBulbTemperature',
 'HourlyPrecipitation',
 'HourlyCloudCoverage',
 'HourlyCloudElevation',
 'HourlyWindSpeed'
]

In [0]:
# drop row if null present in weath_feat_list
train_df = train_df.dropna(subset=weather_feat_list)
validation_df = validation_df.dropna(subset=weather_feat_list)

In [0]:
train_df.columns


### Feature Eng.

#### Was the previous flight delayed? And by how much was the previous flight delayed?

In [0]:
train_df = train_df \
    .withColumn("prev_flight_delay_in_minutes", F.lag("DEP_DELAY_NEW", 1) \
        .over(Window.partitionBy("TAIL_NUM") \
        .orderBy("utc_timestamp"))) \
    .withColumn("prev_flight_delay_in_minutes", F.when(F.col("prev_flight_delay_in_minutes").isNull(), -1) \
        .otherwise(F.col("prev_flight_delay_in_minutes"))) \
    .withColumn("prev_flight_delay", F.when(F.col("prev_flight_delay_in_minutes") > 15, 1) \
        .otherwise(F.lit(0)))
    
validation_df = validation_df \
    .withColumn("prev_flight_delay_in_minutes", F.lag("DEP_DELAY_NEW", 1) \
        .over(Window.partitionBy("TAIL_NUM") \
        .orderBy("utc_timestamp"))) \
    .withColumn("prev_flight_delay_in_minutes", F.when(F.col("prev_flight_delay_in_minutes").isNull(), -1) \
        .otherwise(F.col("prev_flight_delay_in_minutes"))) \
    .withColumn("prev_flight_delay", F.when(F.col("prev_flight_delay_in_minutes") > 15, 1) \
        .otherwise(F.lit(0)))


### [Feature] Delay time for flights at departure locations over the past 7 days

In [0]:
window_7d_origin = Window \
    .partitionBy("ORIGIN_AIRPORT_SEQ_ID") \
    .orderBy(F.col("utc_timestamp").cast("long")) \
    .rangeBetween(-604800, -14400) # -7 days, -4 hours

train_df = train_df.withColumn(
    'delay_origin_7d_sum_raw', 
    F.sum('DEP_DELAY_NEW').over(window_7d_origin)
)

# Handle the nulls by coalescing the raw feature with 0
train_df = train_df.withColumn(
    'delay_origin_7d', 
    F.coalesce(F.col('delay_origin_7d_sum_raw'), F.lit(0))
).drop('delay_origin_7d_sum_raw') 

validation_df = validation_df.withColumn(
    'delay_origin_7d_sum_raw', 
    F.sum('DEP_DELAY_NEW').over(window_7d_origin)
)

# Handle the nulls by coalescing the raw feature with 0
validation_df = validation_df.withColumn(
    'delay_origin_7d', 
    F.coalesce(F.col('delay_origin_7d_sum_raw'), F.lit(0))
).drop('delay_origin_7d_sum_raw') 

### [Feature] Number of delayed flights at departure and carrier location over the last 7 days

In [0]:
window_7d_origin_carrier = Window \
    .partitionBy("ORIGIN_AIRPORT_SEQ_ID", "OP_UNIQUE_CARRIER") \
    .orderBy(F.col("utc_timestamp").cast("long")) \
    .rangeBetween(-604800, -14400) # -7 days, -4 hours

train_df = train_df.withColumn(
    'delay_origin_carrier_7d_raw', 
    F.sum('DEP_DELAY_NEW').over(window_7d_origin_carrier)
)

# Handle the nulls by coalescing the raw feature with 0
train_df = train_df.withColumn(
    'delay_origin_carrier_7d', 
    F.coalesce(F.col('delay_origin_carrier_7d_raw'), F.lit(0))
).drop('delay_origin_carrier_7d_raw') 


validation_df = validation_df.withColumn(
    'delay_origin_carrier_7d_raw', 
    F.sum('DEP_DELAY_NEW').over(window_7d_origin_carrier)
)

# Handle the nulls by coalescing the raw feature with 0
validation_df = validation_df.withColumn(
    'delay_origin_carrier_7d', 
    F.coalesce(F.col('delay_origin_carrier_7d_raw'), F.lit(0))
).drop('delay_origin_carrier_7d_raw') 

### [Feature] number of delays in route in the last 7 days

In [0]:
train_df = train_df.withColumn(
  "route",
  F.concat(F.col("ORIGIN"), F.lit("-"), F.col("DEST"))
)

validation_df = validation_df.withColumn(
  "route",
  F.concat(F.col("ORIGIN"), F.lit("-"), F.col("DEST"))
)

In [0]:
window_7d_route = Window \
    .partitionBy("route") \
    .orderBy(F.col("utc_timestamp").cast("long")) \
    .rangeBetween(-604800, -14400) # -7 days, -4 hours


train_df = train_df.withColumn(
    'delay_route_7d_sum_raw', 
    F.sum('DEP_DELAY_NEW').over(window_7d_route)
)

# Handle the nulls by coalescing the raw feature with 0
train_df = train_df.withColumn(
    'delay_route_7d', 
    F.coalesce(F.col('delay_route_7d_sum_raw'), F.lit(0))
).drop('delay_route_7d_sum_raw') 

validation_df = validation_df.withColumn(
    'delay_route_7d_sum_raw', 
    F.sum('DEP_DELAY_NEW').over(window_7d_route)
)

# Handle the nulls by coalescing the raw feature with 0
validation_df = validation_df.withColumn(
    'delay_route_7d', 
    F.coalesce(F.col('delay_route_7d_sum_raw'), F.lit(0))
).drop('delay_route_7d_sum_raw') 

### [Feature] - number of flights per day for one plane

In [0]:
window_flights_24h = Window \
  .partitionBy("TAIL_NUM", "FL_DATE") \
  .orderBy(F.col("utc_timestamp").cast("long"))

train_df = train_df.withColumn(
    'flight_count_24h', 
    F.count("*").over(window_flights_24h)
)

validation_df = validation_df.withColumn(
    'flight_count_24h', 
    F.count("*").over(window_flights_24h)
)

### [Feature] time between landed and scheduled flight

In [0]:
window_turnaround = Window \
    .partitionBy("TAIL_NUM") \
    .orderBy(F.col("WHEELS_ON").cast("long")) 


train_df = train_df.withColumn(
    "next_scheduled_dep_ts", 
    F.lead("CRS_DEP_TIME", 1).over(window_turnaround)
)

train_df = train_df.withColumn(
    "LANDING_TIME_DIFF_MINUTES",
    F.coalesce(
        (F.col("next_scheduled_dep_ts").cast("long") - F.col("WHEELS_ON").cast("long")) / 60,
        F.lit(-999) 
    )
).drop("next_scheduled_dep_ts")

train_df.select("TAIL_NUM", "WHEELS_ON", "CRS_DEP_TIME", "LANDING_TIME_DIFF_MINUTES").orderBy("TAIL_NUM", "WHEELS_ON").show(5)

In [0]:
validation_df = validation_df.withColumn(
    "next_scheduled_dep_ts", 
    F.lead("CRS_DEP_TIME", 1).over(window_turnaround)
)

validation_df = validation_df.withColumn(
    "LANDING_TIME_DIFF_MINUTES",
    F.coalesce(
        (F.col("next_scheduled_dep_ts").cast("long") - F.col("WHEELS_ON").cast("long")) / 60,
        F.lit(-999) 
    )
).drop("next_scheduled_dep_ts")

### [Feature] Average Delay time by airport
- by origin airport and by destination

In [0]:
avg_delay_by_airport_train = train_df.groupBy("DEST_AIRPORT_SEQ_ID").agg(
    F.avg("ARR_DELAY").alias("AVG_ARR_DELAY")
)

avg_delay_by_airport_val = validation_df.groupBy("DEST_AIRPORT_SEQ_ID").agg(
    F.avg("ARR_DELAY").alias("AVG_ARR_DELAY")
)

In [0]:
# train_df.select("DEST", "ARR_DELAY", "AVG_ARR_DELAY").show(20, False)

In [0]:

window_7d_origin = Window \
    .partitionBy("ORIGIN_AIRPORT_SEQ_ID") \
    .orderBy(F.col("utc_timestamp").cast("long")) \
    .rangeBetween(-604800, -14400) # -7 days (604800s) to -4 hours (14400s)

train_df = train_df.withColumn(
    'avg_delay_origin_7d_raw', 
    F.avg('ARR_DELAY').over(window_7d_origin)
)

train_df = train_df.withColumn(
    'AVG_ARR_DELAY_ORIGIN', 
    F.coalesce(F.col('avg_delay_origin_7d_raw'), F.lit(0))
).drop('avg_delay_origin_7d_raw') 

validation_df = validation_df.withColumn(
    'avg_delay_origin_7d_raw', 
    F.avg('ARR_DELAY').over(window_7d_origin)
)

validation_df = validation_df.withColumn(
    'AVG_ARR_DELAY_ORIGIN', 
    F.coalesce(F.col('avg_delay_origin_7d_raw'), F.lit(0))
).drop('avg_delay_origin_7d_raw')

### [Feature] Average taxi-out time by airport

In [0]:
train_df = train_df.withColumn(
    'avg_taxi_out_origin_7d_raw', 
    F.avg('TAXI_OUT').over(window_7d_origin)
)

train_df = train_df.withColumn(
    'AVG_TAXI_OUT_ORIGIN', 
    F.coalesce(F.col('avg_taxi_out_origin_7d_raw'), F.lit(0))
).drop('avg_taxi_out_origin_7d_raw') 

validation_df = validation_df.withColumn(
    'avg_taxi_out_origin_7d_raw', 
    F.avg('TAXI_OUT').over(window_7d_origin)
)

validation_df = validation_df.withColumn(
    'AVG_TAXI_OUT_ORIGIN', 
    F.coalesce(F.col('avg_taxi_out_origin_7d_raw'), F.lit(0))
).drop('avg_taxi_out_origin_7d_raw')

In [0]:
null_counts = validation_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in validation_df.columns])
display(null_counts)

### Number of delays before in the last 4 hours

In [0]:
useful_columns = [
    "QUARTER",
    "MONTH",
    "YEAR",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "OP_CARRIER",
    "TAIL_NUM",
    "ORIGIN_AIRPORT_SEQ_ID",
    "DEST_AIRPORT_SEQ_ID",
    "CRS_DEP_MINUTES",
    "CRS_ELAPSED_TIME",
    "DISTANCE",
    "DEP_DELAY_NEW",
    "utc_timestamp",
    "prev_flight_delay_in_minutes",
    "prev_flight_delay",
    "origin_delays_4h",
    "delay_origin_7d",
    "delay_origin_carrier_7d",
    "delay_route_7d",
    "flight_count_24h",
    "LANDING_TIME_DIFF_MINUTES",
    "AVG_ARR_DELAY_ORIGIN",
    "AVG_TAXI_OUT_ORIGIN",
    'HourlyDryBulbTemperature',
    'HourlyDewPointTemperature',
    'HourlyRelativeHumidity',
    'HourlyAltimeterSetting',
    'HourlyVisibility',
    'HourlyStationPressure',
    'HourlyWetBulbTemperature',
    'HourlyPrecipitation',
    'HourlyCloudCoverage',
    'HourlyCloudElevation',
    'HourlyWindSpeed'

]

In [0]:
window_4h = Window \
    .partitionBy("ORIGIN_AIRPORT_SEQ_ID") \
    .orderBy(F.col("utc_timestamp").cast("long")) \
    .rangeBetween(-14400, -7200) # 4 hours to 2 hours before

train_df = train_df \
    .withColumn("origin_delays_4h", F.count(F.when(F.col("DEP_DELAY_NEW") > 15, 1)) \
        .over(window_4h)
    )
validation_df = validation_df \
    .withColumn("origin_delays_4h", F.count(F.when(F.col("DEP_DELAY_NEW") > 15, 1)) \
        .over(window_4h)
    )

In [0]:
# train_df.columns

# Feature Selection

In [0]:
# for column_name in baselines_columns:
#     print(f"{column_name} ------> {train_df.filter(F.col(column_name).isNull()).count()}")

In [0]:
baselines_columns = [
    "QUARTER",
    "MONTH",
    "YEAR",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "OP_CARRIER",
    "TAIL_NUM",
    "ORIGIN_AIRPORT_SEQ_ID",
    "DEST_AIRPORT_SEQ_ID",
    "CRS_DEP_MINUTES",
    "CRS_ELAPSED_TIME",
    "DISTANCE",
    "DEP_DELAY_NEW",
    "utc_timestamp",
    "prev_flight_delay_in_minutes",
    "prev_flight_delay",
    "origin_delays_4h",
    "delay_origin_7d",
    "delay_origin_carrier_7d",
    "delay_route_7d",
    "flight_count_24h",
    "LANDING_TIME_DIFF_MINUTES",
    "AVG_ARR_DELAY_ORIGIN",
    "AVG_TAXI_OUT_ORIGIN",
    'HourlyDryBulbTemperature',
    'HourlyDewPointTemperature',
    'HourlyRelativeHumidity',
    'HourlyAltimeterSetting',
    'HourlyVisibility',
    'HourlyStationPressure',
    'HourlyWetBulbTemperature',
    'HourlyPrecipitation',
    'HourlyCloudCoverage',
    'HourlyCloudElevation',
    'HourlyWindSpeed'

]

In [0]:
print(train_df.columns == validation_df.columns)
# print(train_df.columns == baselines_columns)

# for col in baselines_columns:
#     if col not in train_df.columns:
#         print(col)

for col in train_df.columns:
    if col not in validation_df.columns:
        print(col)



In [0]:
train_df = train_df.filter(F.col("DEP_DELAY_NEW").isNotNull()).select(baselines_columns)
validation_df = validation_df.filter(F.col("DEP_DELAY_NEW").isNotNull()).select(baselines_columns)

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor

# Categorical encoding
carrier_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="carrier_idx", handleInvalid="keep")
origin_indexer = StringIndexer(inputCol="ORIGIN_AIRPORT_SEQ_ID", outputCol="origin_idx", handleInvalid="keep")
dest_indexer = StringIndexer(inputCol="DEST_AIRPORT_SEQ_ID", outputCol="dest_idx", handleInvalid="keep")
tail_num_indexer = StringIndexer(inputCol="TAIL_NUM", outputCol="tail_num_idx", handleInvalid="keep")

carrier_encoder = OneHotEncoder(inputCol="carrier_idx", outputCol="carrier_vec")
origin_encoder = OneHotEncoder(inputCol="origin_idx", outputCol="origin_vec")
dest_encoder = OneHotEncoder(inputCol="dest_idx", outputCol="dest_vec")
tail_num_encoder = OneHotEncoder(inputCol="tail_num_idx", outputCol="tail_num_vec")



In [0]:
# Assemble all features
assembler = VectorAssembler(
    inputCols=[
        "QUARTER",
        "MONTH", 
        "YEAR",
        "DAY_OF_MONTH",
        "DAY_OF_WEEK",
        "carrier_vec",
        "origin_vec",
        "dest_vec",
        "tail_num_vec",
        "CRS_DEP_MINUTES",
        "CRS_ELAPSED_TIME",
        "DISTANCE",
        "prev_flight_delay_in_minutes",
        "prev_flight_delay",
        "origin_delays_4h",
        "delay_origin_7d",
        "delay_origin_carrier_7d",
        "delay_route_7d",
        "flight_count_24h",
        "LANDING_TIME_DIFF_MINUTES",
        "AVG_ARR_DELAY_ORIGIN",
        "AVG_TAXI_OUT_ORIGIN",
        'HourlyDryBulbTemperature',
        'HourlyDewPointTemperature',
        'HourlyRelativeHumidity',
        'HourlyAltimeterSetting',
        'HourlyVisibility',
        'HourlyStationPressure',
        'HourlyWetBulbTemperature',
        'HourlyPrecipitation',
        'HourlyCloudCoverage',
        'HourlyCloudElevation',
        'HourlyWindSpeed'

    ],
    outputCol="features"
)

# Model

In [0]:
# baseline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from mlflow.models import infer_signature

mlflow.spark.autolog()
with mlflow.start_run(run_name="LR-1y_baseline_pipeline_check"):
    MODEL_NAME = "LR_1y_BASELINE_CHECK"

    linear_reg = LinearRegression(
        featuresCol="features",
        labelCol="DEP_DELAY_NEW",
        # Linear Regression has different parameters than Random Forest
        maxIter=10, 
        regParam=0.3
    )
    # rf = RandomForestRegressor(
    #     featuresCol="features",  
    #     labelCol="DEP_DELAY_NEW",   
    #     numTrees=20,
    #     maxDepth=10
    # )

    # Create pipeline
    # pipeline = Pipeline(stages=[
    #     carrier_indexer, origin_indexer, dest_indexer, 
    #     carrier_encoder, origin_encoder, dest_encoder, 
    #     assembler,
    #     # linear_reg
    #     rf
    # ])

    pipeline = Pipeline(stages=[
        carrier_indexer, origin_indexer, dest_indexer, tail_num_indexer,
        carrier_encoder, origin_encoder, dest_encoder, tail_num_encoder,
        assembler,
        linear_reg
        # rf
    ])

    model = pipeline.fit(train_df)
    training_predictions = model.transform(train_df)
    validation_predictions = model.transform(validation_df)

    mae_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="mae"           
    )

    rmse_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="rmse"
    )

    # Calculate MAE
    mae_t = mae_evaluator.evaluate(training_predictions)
    mae_v = mae_evaluator.evaluate(validation_predictions)
    # Calculate RMSE
    rmse_t = rmse_evaluator.evaluate(training_predictions)
    rmse_v = rmse_evaluator.evaluate(validation_predictions)

    signature = infer_signature(train_df, training_predictions)

    mlflow.spark.log_model(
        model, 
        MODEL_NAME,
        input_example=train_df.limit(1).toPandas(),
        signature=signature,
        registered_model_name="flight_delay_prediction_baseline"
        )

    mlflow.log_metric("train_mae", mae_t)
    mlflow.log_metric("validation_mae", mae_v)
    mlflow.log_metric("train_rmse", rmse_t)
    mlflow.log_metric("validation_rmse", rmse_v)


## Checkpoint results to MLflow?

## Develop and Run Pipeline

## Cross Validation

## Save metrics, pipeline, any other steps

## Feature Engineering Ideas
- Previous flight delay in minutes for the aircraft [DONE] - this added value to the linear regression model!
- Number of delayed flights from 4 hours (DONE)
- Number of delays in the route in the last 30 days
- Time between landing and scheduled current flight
- Airport + utc time type of delay - Ohare at 6PM is always late
- Number of delayed flights in departure and arrival location (total or 4 hours before, 6 hours before, etc.)
- Average delay time by airport
- Average taxi out time by airport/flight